#Dataset

In [19]:
import cv2 # OpenCV
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
import os
import tensorflow_datasets as tfds
import pandas as pd


# Load Dataset

In [4]:
path = "FULL"


PAKE KERAS

In [5]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   validation_split=0.5)
training_set = train_datagen.flow_from_directory(path+"/train",
                                                batch_size=64,
                                                target_size=(224,224),
                                                shuffle=True,
                                                class_mode='categorical',
                                                subset='training')

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True)
validation_set = train_datagen.flow_from_directory(path+"/test",
                                                batch_size=64,
                                                target_size=(224,224),
                                                shuffle=True,

                                                class_mode='categorical',
                                                subset='training')

Found 15923 images belonging to 7 classes.
Found 7951 images belonging to 9 classes.


# Model

## Model FineTuning

In [6]:
VGG19=tf.keras.applications.vgg19.VGG19(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None
)
VGG19.trainable = True
print(len(VGG19.layers))

22


In [7]:
tf.keras.utils.plot_model(VGG19, show_shapes=True, dpi=76, to_file='model_v1.png')

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [8]:
i = 0
for layer in VGG19.layers:
  if i > 12:
    layer.trainable = False
  i+=1

In [9]:
# Data Augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.2),  # Rotate up to 20 degrees
    tf.keras.layers.RandomTranslation(0.1, 0.15),  # Translate 10% width, 15% height
    tf.keras.layers.RandomBrightness(factor=[0.2, 1.0]),  # Change brightness between 0.2 and 1
    tf.keras.layers.RandomZoom(height_factor=(-0.1, 0.2), width_factor=(-0.1, 0.2)),  # Zoom out up to 10%, zoom in up to 20%
    tf.keras.layers.RandomFlip("horizontal"),  # Flip horizontally
])

input = tf.keras.layers.Input(shape=(224, 224, 3))
# Apply data augmentation to the input
x = data_augmentation(input)
x = VGG19(input, training=False)
x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(7, activation = 'softmax')(x)
VGG19 = tf.keras.models.Model(inputs = input, outputs = x, name = "VGG19")

In [10]:
VGG19.summary()

Model: "VGG19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg19 (Functional)              │ (None, 7, 7, 512)      │    20,024,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 4, 4, 512)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 7)              │        57,351 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,081,735 (76.61 MB)

 Trainable params: 3,563,079 (13.59 MB)

 Non-trainable params: 16,518,656 (63.01 MB)

## Model Reverse

In [27]:
VGG19=tf.keras.applications.vgg19.VGG19(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None
)
VGG19.trainable = True
print(len(VGG19.layers))

22


In [28]:
i = 0
for layer in VGG19.layers:
  if i < 12:
    layer.trainable = False
  i+=1

In [29]:
# Data Augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.2),  # Rotate up to 20 degrees
    tf.keras.layers.RandomTranslation(0.1, 0.15),  # Translate 10% width, 15% height
    tf.keras.layers.RandomBrightness(factor=[0.2, 1.0]),  # Change brightness between 0.2 and 1
    tf.keras.layers.RandomZoom(height_factor=(-0.1, 0.2), width_factor=(-0.1, 0.2)),  # Zoom out up to 10%, zoom in up to 20%
    tf.keras.layers.RandomFlip("horizontal"),  # Flip horizontally
])

input = tf.keras.layers.Input(shape=(224, 224, 3))
# Apply data augmentation to the input
x = data_augmentation(input)
x = VGG19(input, training=False)
x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(7, activation = 'softmax')(x)
VGG19 = tf.keras.models.Model(inputs = input, outputs = x, name = "VGG19")

VGG19.summary()

Model: "VGG19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg19 (Functional)              │ (None, 7, 7, 512)      │    20,024,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 4, 4, 512)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │        57,351 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,081,735 (76.61 MB)

 Trainable params: 17,756,167 (67.73 MB)

 Non-trainable params: 2,325,568 (8.87 MB)

## Model Full Freeze`

In [31]:
VGG19=tf.keras.applications.vgg19.VGG19(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None
)
VGG19.trainable = False
print(len(VGG19.layers))

22


In [32]:
# Data Augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.2),  # Rotate up to 20 degrees
    tf.keras.layers.RandomTranslation(0.1, 0.15),  # Translate 10% width, 15% height
    tf.keras.layers.RandomBrightness(factor=[0.2, 1.0]),  # Change brightness between 0.2 and 1
    tf.keras.layers.RandomZoom(height_factor=(-0.1, 0.2), width_factor=(-0.1, 0.2)),  # Zoom out up to 10%, zoom in up to 20%
    tf.keras.layers.RandomFlip("horizontal"),  # Flip horizontally
])

input = tf.keras.layers.Input(shape=(224, 224, 3))
# Apply data augmentation to the input
x = data_augmentation(input)
x = VGG19(input, training=False)
x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(7, activation = 'softmax')(x)
VGG19 = tf.keras.models.Model(inputs = input, outputs = x, name = "VGG19")

VGG19.summary()

Model: "VGG19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg19 (Functional)              │ (None, 7, 7, 512)      │    20,024,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_2             │ (None, 4, 4, 512)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 7)              │        57,351 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,081,735 (76.61 MB)

 Trainable params: 57,351 (224.03 KB)

 Non-trainable params: 20,024,384 (76.39 MB)

# Training

## Train Fine Tuning

In [10]:
steps_per_epoch = training_set.n // 128
validation_steps = validation_set.n // 128

VGG19.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = VGG19.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=20,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )

Epoch 1/20
124/124 ━━━━━━━━━━━━━━━━━━━━ 1860s 15s/step - accuracy: 0.1937 - loss: 1.9373 - val_accuracy: 0.2445 - val_loss: 1.8200
Epoch 2/20
124/124 ━━━━━━━━━━━━━━━━━━━━ 1719s 14s/step - accuracy: 0.2930 - loss: 1.7637 - val_accuracy: 0.3695 - val_loss: 1.6324
Epoch 3/20
  1/124 ━━━━━━━━━━━━━━━━━━━━ 25:38 13s/step - accuracy: 0.3438 - loss: 1.6477

c:\Users\mario\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


124/124 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.3438 - loss: 1.6477 - val_accuracy: 0.3750 - val_loss: 1.6467
Epoch 4/20
124/124 ━━━━━━━━━━━━━━━━━━━━ 1726s 14s/step - accuracy: 0.3741 - loss: 1.6175 - val_accuracy: 0.4083 - val_loss: 1.5545
Epoch 5/20
124/124 ━━━━━━━━━━━━━━━━━━━━ 1709s 14s/step - accuracy: 0.4006 - loss: 1.5374 - val_accuracy: 0.4194 - val_loss: 1.4709
Epoch 6/20
124/124 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.4531 - loss: 1.4273 - val_accuracy: 0.5000 - val_loss: 1.4324
Epoch 7/20
124/124 ━━━━━━━━━━━━━━━━━━━━ 1717s 14s/step - accuracy: 0.4263 - loss: 1.4951 - val_accuracy: 0.4481 - val_loss: 1.4576
Epoch 8/20
124/124 ━━━━━━━━━━━━━━━━━━━━ 1708s 14s/step - accuracy: 0.4372 - loss: 1.4464 - val_accuracy: 0.4738 - val_loss: 1.3916
Epoch 9/20
124/124 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.6094 - loss: 1.1915 - val_accuracy: 0.3750 - val_loss: 1.6294
Epoch 10/20
124/124 ━━━━━━━━━━━━━━━━━━━━ 1710s 14s/step - accuracy: 0.4641 - loss: 1.3976 - val_accu

In [6]:
steps_per_epoch = training_set.n // 128
validation_steps = validation_set.n // 128

VGG19.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = VGG19.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=20,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )

Epoch 1/20


c:\Users\mario\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


124/124 ━━━━━━━━━━━━━━━━━━━━ 1838s 15s/step - accuracy: 0.5220 - loss: 1.2563 - val_accuracy: 0.5136 - val_loss: 1.2952
Epoch 2/20
124/124 ━━━━━━━━━━━━━━━━━━━━ 1793s 14s/step - accuracy: 0.5362 - loss: 1.2065 - val_accuracy: 0.5202 - val_loss: 1.2672
Epoch 3/20
  1/124 ━━━━━━━━━━━━━━━━━━━━ 26:07 13s/step - accuracy: 0.6250 - loss: 1.0715

c:\Users\mario\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


124/124 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.6250 - loss: 1.0715 - val_accuracy: 0.5000 - val_loss: 1.3220
Epoch 4/20
124/124 ━━━━━━━━━━━━━━━━━━━━ 1795s 14s/step - accuracy: 0.5400 - loss: 1.2025 - val_accuracy: 0.5257 - val_loss: 1.2644
Epoch 5/20
124/124 ━━━━━━━━━━━━━━━━━━━━ 1796s 14s/step - accuracy: 0.5528 - loss: 1.1763 - val_accuracy: 0.5328 - val_loss: 1.2417
Epoch 6/20
124/124 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.4688 - loss: 1.2855 - val_accuracy: 0.5000 - val_loss: 0.9645
Epoch 7/20
124/124 ━━━━━━━━━━━━━━━━━━━━ 1794s 14s/step - accuracy: 0.5527 - loss: 1.1635 - val_accuracy: 0.5086 - val_loss: 1.2792
Epoch 8/20
124/124 ━━━━━━━━━━━━━━━━━━━━ 1792s 14s/step - accuracy: 0.5854 - loss: 1.1137 - val_accuracy: 0.5398 - val_loss: 1.2602
Epoch 9/20
124/124 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step - accuracy: 0.5469 - loss: 1.3076 - val_accuracy: 0.2500 - val_loss: 1.5526
Epoch 10/20
124/124 ━━━━━━━━━━━━━━━━━━━━ 1795s 14s/step - accuracy: 0.5558 - loss: 1.1422 - val_acc

In [6]:
steps_per_epoch = training_set.n // 128
validation_steps = validation_set.n // 128

VGG19.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = VGG19.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=30,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )

Epoch 1/30


c:\Users\mario\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


124/124 ━━━━━━━━━━━━━━━━━━━━ 1765s 14s/step - accuracy: 0.6293 - loss: 0.9901 - val_accuracy: 0.5479 - val_loss: 1.2009
Epoch 2/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1710s 14s/step - accuracy: 0.6472 - loss: 0.9408 - val_accuracy: 0.5726 - val_loss: 1.2101
Epoch 3/30
  1/124 ━━━━━━━━━━━━━━━━━━━━ 25:01 12s/step - accuracy: 0.6562 - loss: 1.1939

c:\Users\mario\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


124/124 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.6562 - loss: 1.1939 - val_accuracy: 0.1250 - val_loss: 1.9086
Epoch 4/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1715s 14s/step - accuracy: 0.6489 - loss: 0.9366 - val_accuracy: 0.5519 - val_loss: 1.2540
Epoch 5/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1697s 14s/step - accuracy: 0.6759 - loss: 0.8964 - val_accuracy: 0.5237 - val_loss: 1.3320
Epoch 6/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.5938 - loss: 1.0752 - val_accuracy: 0.5000 - val_loss: 1.3454
Epoch 7/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1701s 14s/step - accuracy: 0.6677 - loss: 0.8922 - val_accuracy: 0.5383 - val_loss: 1.2445
Epoch 8/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1693s 14s/step - accuracy: 0.6743 - loss: 0.8811 - val_accuracy: 0.5655 - val_loss: 1.2126
Epoch 9/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.6406 - loss: 0.9368 - val_accuracy: 0.7500 - val_loss: 1.0240
Epoch 10/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1704s 14s/step - accuracy: 0.6829 - loss: 0.8545 - val_accu

In [19]:
steps_per_epoch = training_set.n // 128
validation_steps = validation_set.n // 128

VGG19.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = VGG19.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=30,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps
                    )

Epoch 1/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1727s 14s/step - accuracy: 0.7626 - loss: 0.6523 - val_accuracy: 0.5539 - val_loss: 1.4437
Epoch 2/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1710s 14s/step - accuracy: 0.7707 - loss: 0.6444 - val_accuracy: 0.5484 - val_loss: 1.5275
Epoch 3/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.7031 - loss: 0.8083 - val_accuracy: 0.8750 - val_loss: 1.2369
Epoch 4/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1724s 14s/step - accuracy: 0.7876 - loss: 0.5968 - val_accuracy: 0.5474 - val_loss: 1.5316
Epoch 5/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1732s 14s/step - accuracy: 0.7728 - loss: 0.6387 - val_accuracy: 0.5549 - val_loss: 1.4503
Epoch 6/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.7969 - loss: 0.5120 - val_accuracy: 0.5000 - val_loss: 1.6485
Epoch 7/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1743s 14s/step - accuracy: 0.7926 - loss: 0.5846 - val_accuracy: 0.5368 - val_loss: 1.5861
Epoch 8/30
124/124 ━━━━━━━━━━━━━━━━━━━━ 1743s 14s/step - accuracy: 0.7908 - loss: 0.601

In [11]:
VGG19.save('VGG19Augmented_TrainableBanyak1.keras')

In [7]:
VGG19.save('VGG19Augmented_TrainableBanyak2.keras')


In [7]:
VGG19.save('VGG19Augmented_TrainableBanyak3.keras')


In [20]:
VGG19.save('VGG19Augmented_TrainableBanyak4.keras')


In [16]:
VGG19.save('VGG19Augmented_TrainableBanyakBackup.keras')


## Train Reverse

In [16]:
steps_per_epoch = training_set.n // 128
validation_steps = validation_set.n // 128

VGG19.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = VGG19.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=10,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps                                                                                                                                                                                                                                                                                                                                                                                                                      
                    )


Epoch 1/10


c:\Users\mario\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


124/124 ━━━━━━━━━━━━━━━━━━━━ 5393s 43s/step - accuracy: 0.2184 - loss: 1.9236 - val_accuracy: 0.2661 - val_loss: 1.8192
Epoch 2/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 5473s 44s/step - accuracy: 0.2907 - loss: 1.7909 - val_accuracy: 0.3745 - val_loss: 1.6775
Epoch 3/10
  1/124 ━━━━━━━━━━━━━━━━━━━━ 1:24:13 41s/step - accuracy: 0.2969 - loss: 1.8096

c:\Users\mario\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


124/124 ━━━━━━━━━━━━━━━━━━━━ 44s 21ms/step - accuracy: 0.2969 - loss: 1.8096 - val_accuracy: 0.2500 - val_loss: 2.0590
Epoch 4/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 5633s 45s/step - accuracy: 0.3708 - loss: 1.6339 - val_accuracy: 0.3926 - val_loss: 1.5575
Epoch 5/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 5610s 45s/step - accuracy: 0.4062 - loss: 1.5497 - val_accuracy: 0.4022 - val_loss: 1.5417
Epoch 6/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 42s 19ms/step - accuracy: 0.5156 - loss: 1.4401 - val_accuracy: 0.3750 - val_loss: 1.3083
Epoch 7/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 5490s 44s/step - accuracy: 0.4206 - loss: 1.5140 - val_accuracy: 0.4199 - val_loss: 1.5235
Epoch 8/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 5443s 44s/step - accuracy: 0.4162 - loss: 1.5109 - val_accuracy: 0.4335 - val_loss: 1.5034
Epoch 9/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 41s 15ms/step - accuracy: 0.4375 - loss: 1.4062 - val_accuracy: 0.2500 - val_loss: 1.8196
Epoch 10/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 5409s 44s/step - accuracy: 0.4639 - loss: 1.4259 - val_a

In [17]:
VGG19.save('VGG19Augmented_TrainableDikit1.keras')

In [18]:
steps_per_epoch = training_set.n // 128
validation_steps = validation_set.n // 128

VGG19.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = VGG19.fit(x=training_set,
                    validation_data=validation_set,
                    epochs=10,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps                                                                                                                                                                                                                                                                                                                                                                                                                      
                    )


Epoch 1/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 5448s 44s/step - accuracy: 0.4389 - loss: 1.4576 - val_accuracy: 0.4435 - val_loss: 1.4471
Epoch 2/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 5473s 44s/step - accuracy: 0.4585 - loss: 1.4101 - val_accuracy: 0.4551 - val_loss: 1.4471
Epoch 3/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 43s 15ms/step - accuracy: 0.4688 - loss: 1.4588 - val_accuracy: 0.3750 - val_loss: 1.5933
Epoch 4/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 5378s 43s/step - accuracy: 0.4770 - loss: 1.3879 - val_accuracy: 0.4541 - val_loss: 1.4289
Epoch 5/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 5364s 43s/step - accuracy: 0.4925 - loss: 1.3397 - val_accuracy: 0.4662 - val_loss: 1.4473
Epoch 6/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 42s 20ms/step - accuracy: 0.5312 - loss: 1.3464 - val_accuracy: 0.3750 - val_loss: 1.3065
Epoch 7/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 5305s 43s/step - accuracy: 0.4964 - loss: 1.3438 - val_accuracy: 0.4914 - val_loss: 1.3728
Epoch 8/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 5271s 43s/step - accuracy: 0.4922 - loss: 1.3

In [19]:
VGG19.save('VGG19Augmented_TrainableDikit2.keras')

In [8]:
steps_per_epoch = training_set.n // 128
validation_steps = validation_set.n // 128

VGG19.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = VGG19.fit(x=training_set,
                    # validation_data=validation_set,
                    epochs=10,
                    steps_per_epoch=steps_per_epoch,
                    # validation_steps=validation_steps                                                                                                                                                                                                                                                                                                                                                                                                                      
                    )


Epoch 1/10


c:\Users\mario\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


124/124 ━━━━━━━━━━━━━━━━━━━━ 4452s 36s/step - accuracy: 0.5138 - loss: 1.2910
Epoch 2/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 4391s 35s/step - accuracy: 0.5308 - loss: 1.2576
Epoch 3/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 38s 6ms/step - accuracy: 0.5312 - loss: 1.2969    
Epoch 4/10


c:\Users\mario\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


124/124 ━━━━━━━━━━━━━━━━━━━━ 4387s 35s/step - accuracy: 0.5437 - loss: 1.2125
Epoch 5/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 4333s 35s/step - accuracy: 0.5409 - loss: 1.2148
Epoch 6/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 34s 3ms/step - accuracy: 0.6094 - loss: 0.9438    
Epoch 7/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 4349s 35s/step - accuracy: 0.5678 - loss: 1.1748
Epoch 8/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 4447s 36s/step - accuracy: 0.5603 - loss: 1.1768
Epoch 9/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 35s 3ms/step - accuracy: 0.5625 - loss: 1.1806    
Epoch 10/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 4585s 37s/step - accuracy: 0.5787 - loss: 1.1602


In [9]:
VGG19.save('VGG19Augmented_TrainableDikit3.keras')

In [7]:
steps_per_epoch = training_set.n // 128
validation_steps = validation_set.n // 128

VGG19.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = VGG19.fit(x=training_set,
                    # validation_data=validation_set,
                    epochs=21,
                    steps_per_epoch=steps_per_epoch,
                    # validation_steps=validation_steps                                                                                                                                                                                                                                                                                                                                                                                                                      
                    )


Epoch 1/21
124/124 ━━━━━━━━━━━━━━━━━━━━ 4490s 36s/step - accuracy: 0.5630 - loss: 1.1567
Epoch 2/21
124/124 ━━━━━━━━━━━━━━━━━━━━ 4503s 36s/step - accuracy: 0.5833 - loss: 1.1240
Epoch 3/21
124/124 ━━━━━━━━━━━━━━━━━━━━ 35s 5ms/step - accuracy: 0.6719 - loss: 0.8340    
Epoch 4/21


c:\Users\mario\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


124/124 ━━━━━━━━━━━━━━━━━━━━ 4361s 35s/step - accuracy: 0.5732 - loss: 1.1255
Epoch 5/21
124/124 ━━━━━━━━━━━━━━━━━━━━ 4515s 36s/step - accuracy: 0.5871 - loss: 1.1056
Epoch 6/21
124/124 ━━━━━━━━━━━━━━━━━━━━ 36s 4ms/step - accuracy: 0.4844 - loss: 1.1538    
Epoch 7/21
124/124 ━━━━━━━━━━━━━━━━━━━━ 4541s 37s/step - accuracy: 0.5962 - loss: 1.0596
Epoch 8/21
124/124 ━━━━━━━━━━━━━━━━━━━━ 4626s 37s/step - accuracy: 0.6029 - loss: 1.0773
Epoch 9/21
124/124 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step - accuracy: 0.5625 - loss: 1.0696    
Epoch 10/21
124/124 ━━━━━━━━━━━━━━━━━━━━ 4668s 38s/step - accuracy: 0.6156 - loss: 1.0341
Epoch 11/21
124/124 ━━━━━━━━━━━━━━━━━━━━ 4670s 38s/step - accuracy: 0.6218 - loss: 1.0305
Epoch 12/21
124/124 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step - accuracy: 0.6719 - loss: 1.0498    
Epoch 13/21
124/124 ━━━━━━━━━━━━━━━━━━━━ 4697s 38s/step - accuracy: 0.6216 - loss: 1.0227
Epoch 14/21
124/124 ━━━━━━━━━━━━━━━━━━━━ 4788s 39s/step - accuracy: 0.6376 - loss: 0.9860
Epoch 15/21
124/124 ━

In [8]:
VGG19.save('VGG19Augmented_TrainableDikit4.keras')

In [9]:
steps_per_epoch = training_set.n // 128
validation_steps = validation_set.n // 128

VGG19.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = VGG19.fit(x=training_set,
                    # validation_data=validation_set,
                    epochs=15,
                    steps_per_epoch=steps_per_epoch,
                    # validation_steps=validation_steps                                                                                                                                                                                                                                                                                                                                                                                                                      
                    )


Epoch 1/15
124/124 ━━━━━━━━━━━━━━━━━━━━ 5010s 40s/step - accuracy: 0.6527 - loss: 0.9488
Epoch 2/15
124/124 ━━━━━━━━━━━━━━━━━━━━ 4962s 40s/step - accuracy: 0.6477 - loss: 0.9514
Epoch 3/15
124/124 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - accuracy: 0.6406 - loss: 1.0763    
Epoch 4/15
124/124 ━━━━━━━━━━━━━━━━━━━━ 4814s 39s/step - accuracy: 0.6397 - loss: 0.9373
Epoch 5/15
124/124 ━━━━━━━━━━━━━━━━━━━━ 4889s 39s/step - accuracy: 0.6706 - loss: 0.9033
Epoch 6/15
124/124 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.5938 - loss: 1.0578    
Epoch 7/15
124/124 ━━━━━━━━━━━━━━━━━━━━ 5004s 40s/step - accuracy: 0.6716 - loss: 0.8869
Epoch 8/15
124/124 ━━━━━━━━━━━━━━━━━━━━ 4954s 40s/step - accuracy: 0.6794 - loss: 0.8702
Epoch 9/15
124/124 ━━━━━━━━━━━━━━━━━━━━ 39s 3ms/step - accuracy: 0.7031 - loss: 0.8580    
Epoch 10/15
124/124 ━━━━━━━━━━━━━━━━━━━━ 4898s 39s/step - accuracy: 0.6863 - loss: 0.8543
Epoch 11/15
124/124 ━━━━━━━━━━━━━━━━━━━━ 4905s 40s/step - accuracy: 0.6834 - loss: 0.8590
Epoch 12/15
1

In [10]:
VGG19.save('VGG19Augmented_TrainableDikit5.keras')

In [11]:
steps_per_epoch = training_set.n // 128
validation_steps = validation_set.n // 128

VGG19.compile(optimizer = 'adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']) 

batch_size = 40
history = VGG19.fit(x=training_set,
                    # validation_data=validation_set,
                    epochs=3,
                    steps_per_epoch=steps_per_epoch,
                    # validation_steps=validation_steps                                                                                                                                                                                                                                                                                                                                                                                                                      
                    )


Epoch 1/3
124/124 ━━━━━━━━━━━━━━━━━━━━ 4467s 36s/step - accuracy: 0.6975 - loss: 0.8103
Epoch 2/3
124/124 ━━━━━━━━━━━━━━━━━━━━ 4446s 36s/step - accuracy: 0.7138 - loss: 0.7910
Epoch 3/3
124/124 ━━━━━━━━━━━━━━━━━━━━ 38s 4ms/step - accuracy: 0.7188 - loss: 0.8244    


In [12]:
VGG19.save('VGG19Augmented_TrainableDikit6.keras')

## Train Freeze (100 epoch, tidak kesave)

In [20]:
VGG19freeze1 = tf.keras.models.load_model('VGG19Augmented_Freeze1.keras')

In [13]:
VGG19freeze2 = tf.keras.models.load_model('VGG19Augmented_Freeze2.keras')

In [14]:
VGG19freeze3 = tf.keras.models.load_model('VGG19Augmented_Freeze3.keras')

In [24]:
VGG19_FullFreeze = tf.keras.models.load_model('VGG19Augmented_Freeze4.keras')

In [ ]:
VGG19_FullFreeze.evaluate(training_set)

c:\Users\mario\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


249/249 ━━━━━━━━━━━━━━━━━━━━ 1874s 8s/step - accuracy: 0.6684 - loss: 0.9425


[0.9451608657836914, 0.6618727445602417]